# Imports and Loading the Model

In [ ]:
import time

import cv2
import torch
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

from indiscapes_dataset import categories_list

In [ ]:
cfg = get_cfg()

cfg.merge_from_file("configs/mask_rcnn_R_50_FPN_1x_dconv_c3-c5_wideanc1.yaml")
cfg.MODEL.WEIGHTS = "model_final.pth"
predictor = DefaultPredictor(cfg)

# Predicting on the Image

In [ ]:
im = cv2.imread("test.jpg")

In [ ]:
cv2.imshow("im", im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
tick = time.time()
outputs = predictor(im)
tock = time.time()
td = tock - tick

In [ ]:
type(outputs)

# Visualizing the Outputs

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow("pred", out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Structurize the JSON

In [ ]:
regions = list()

In [ ]:
all_points = torch.nonzero(outputs['instances'].pred_masks==True)
all_points[torch.where(all_points[:,0] == 1)][:,1:].cpu().tolist()

In [ ]:
regions = list()
for i in range(len(outputs['instances'])):
    regions.append({
        "class_ids": [categories_list[outputs['instances'].pred_classes[i]]],
        "flag": 2,
        "exec_time": td,
        "url": "test.jpg",
        "error": 0,
        "points": all_points[torch.where(all_points[:,0] == i)][:,1:].cpu().tolist()
    })

In [ ]:
output = {
    'regions':regions
}

In [ ]:
import json

with open('result.json', 'w') as fp:
    json.dump(output, fp)